In [10]:
from sentence_transformers import SentenceTransformer
sentences = ["This is an example sentence", "Each sentence is converted"]

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
embeddings = model.encode(sentences, verbose=False)
print(embeddings)

/home/vasim/.local/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


[[ 6.76569194e-02  6.34959489e-02  4.87130918e-02  7.93049783e-02
   3.74480747e-02  2.65276874e-03  3.93749774e-02 -7.09845079e-03
   5.93614094e-02  3.15370336e-02  6.00981005e-02 -5.29052056e-02
   4.06067669e-02 -2.59308796e-02  2.98428107e-02  1.12689147e-03
   7.35149160e-02 -5.03818430e-02 -1.22386597e-01  2.37028003e-02
   2.97265332e-02  4.24768515e-02  2.56337710e-02  1.99513510e-03
  -5.69190867e-02 -2.71597803e-02 -3.29035632e-02  6.60248846e-02
   1.19007207e-01 -4.58791181e-02 -7.26214796e-02 -3.25840078e-02
   5.23413271e-02  4.50553335e-02  8.25297181e-03  3.67024168e-02
  -1.39415516e-02  6.53918535e-02 -2.64271945e-02  2.06362602e-04
  -1.36643825e-02 -3.62810567e-02 -1.95043832e-02 -2.89738029e-02
   3.94270532e-02 -8.84090811e-02  2.62423395e-03  1.36713712e-02
   4.83062640e-02 -3.11565958e-02 -1.17329188e-01 -5.11690564e-02
  -8.85287598e-02 -2.18963306e-02  1.42986402e-02  4.44167927e-02
  -1.34815881e-02  7.43392408e-02  2.66382471e-02 -1.98762808e-02
   1.79191

In [11]:
from langchain_huggingface import HuggingFaceEmbeddings

# model_name = "sentence-transformers/all-mpnet-base-v2"
model_name = "sentence-transformers/all-MiniLM-L6-v2"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

In [38]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

file_path = "/home/vasim/Downloads/PDFs/M3-R4 Programming and Problem Solving through C.pdf"
loader = PyPDFLoader(file_path)

In [39]:
documents = loader.load()
len(documents)

16

In [40]:
documents[0]

Document(metadata={'producer': 'PyPDF', 'creator': 'Nitro PDF Professional', 'creationdate': '', 'moddate': '2016-05-19T17:37:54+05:30', 'source': '/home/vasim/Downloads/PDFs/M3-R4 Programming and Problem Solving through C.pdf', 'total_pages': 16, 'page': 0, 'page_label': '1'}, page_content='48  \n \nM3-R4: PROGRAMMING AND PROBLEM SOLVING THROUGH ‘C’ LANGUAGE  \n \n  \nObjective of the Course \nThe objectives of this course are to make the stude nt understand programming language, \nprogramming, concepts of Loops, reading a set of Data, stepwise refinement, Functions, Control \nstructure, Arrays. After completion of this course the student is expected to analyze the real \nlife problem and write a program in ‘C’ language to solve the problem. The main emphasis of \nthe course will be on problem solving aspect i.e. developing proper algorithms.  \n  \nAfter completion of the course the student will be able to \n•  Develop efficient algorithms for solving a problem . \n•  Use the various

In [41]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
documents = text_splitter.split_documents(documents)

In [42]:
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS

vector_store = FAISS.from_documents(documents, embeddings)

In [43]:
retriever = vector_store.as_retriever(search_type="mmr", search_kwargs={"k": 4})

In [44]:
retriever.invoke("Books to read")[0].page_content

'50  \n \nRECOMMENDED BOOKS  \nMAIN READING  \n \n1. Byron S Gottfried “Programming with C” Second ed ition, Tata McGrawhill, 2007 \n(Paper back) \n2. R.G. Dromey, “How to solve it by Computer”, Pear son Education, 2008. \n3. Kanetkar Y, “Let us C”, BPB Publications, 2007. \n4. Hanly J R & Koffman E.B, “Problem Solving and Programm design in C”, Pearson \nEducation, 2009. \n \nSUPPLEMENTARY READING \n \n1. E. Balagurusamy, “Programming with ANSI-C”, Fourth Edition,2008, Tata McGraw Hill. \n2. Venugopal K. R and Prasad S. R, “Mastering ‘C’”,  Third Edition, 2008, Tata McGraw \nHill. \n3. B.W. Kernighan & D. M. Ritchie, “The C Programming Language”, Second Edition, 2001, \nPearson Education \n4. ISRD Group, “Programming and Problem Solving Using C”, Tata McGraw Hill,2008. \n5. Pradip Dey , Manas Ghosh, “Programming in C”, Ox ford University Press, 2007.'

In [47]:
retriever.invoke("Books to read")

[Document(id='428be3dd-07a7-4851-9b49-46da7b7699a7', metadata={'producer': 'PyPDF', 'creator': 'Nitro PDF Professional', 'creationdate': '', 'moddate': '2016-05-19T17:37:54+05:30', 'source': '/home/vasim/Downloads/PDFs/M3-R4 Programming and Problem Solving through C.pdf', 'total_pages': 16, 'page': 2, 'page_label': '3'}, page_content='50  \n \nRECOMMENDED BOOKS  \nMAIN READING  \n \n1. Byron S Gottfried “Programming with C” Second ed ition, Tata McGrawhill, 2007 \n(Paper back) \n2. R.G. Dromey, “How to solve it by Computer”, Pear son Education, 2008. \n3. Kanetkar Y, “Let us C”, BPB Publications, 2007. \n4. Hanly J R & Koffman E.B, “Problem Solving and Programm design in C”, Pearson \nEducation, 2009. \n \nSUPPLEMENTARY READING \n \n1. E. Balagurusamy, “Programming with ANSI-C”, Fourth Edition,2008, Tata McGraw Hill. \n2. Venugopal K. R and Prasad S. R, “Mastering ‘C’”,  Third Edition, 2008, Tata McGraw \nHill. \n3. B.W. Kernighan & D. M. Ritchie, “The C Programming Language”, Second E

In [46]:
len(retriever.invoke("Books to read")[0].page_content)

842

In [45]:
842

842

In [51]:
def get_retriever(
    file_path: str,
    model_name: str = "sentence-transformers/all-MiniLM-L6-v2",
    chunk_size: int = 1000,
    chunk_overlap: int = 200,
    search_type: str = "mmr",
    search_kwargs: dict = {"k": 1},
) -> FAISS:
    """Load a PDF file and create a retriever using FAISS."""
    loader = PyPDFLoader(file_path)
    documents = loader.load()
    
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    documents = text_splitter.split_documents(documents)

    embeddings = HuggingFaceEmbeddings(model_name=model_name)
    vector_store = FAISS.from_documents(documents, embeddings)
    
    retriever = vector_store.as_retriever(search_type=search_type, search_kwargs=search_kwargs)
    
    return retriever

In [52]:
retriever = get_retriever(file_path)

/home/vasim/.local/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [53]:
retriever.invoke("What is the book about?")

[Document(id='8f71741b-c142-45b9-a056-08649af2058d', metadata={'producer': 'PyPDF', 'creator': 'Nitro PDF Professional', 'creationdate': '', 'moddate': '2016-05-19T17:37:54+05:30', 'source': '/home/vasim/Downloads/PDFs/M3-R4 Programming and Problem Solving through C.pdf', 'total_pages': 16, 'page': 2, 'page_label': '3'}, page_content='50  \n \nRECOMMENDED BOOKS  \nMAIN READING  \n \n1. Byron S Gottfried “Programming with C” Second ed ition, Tata McGrawhill, 2007 \n(Paper back) \n2. R.G. Dromey, “How to solve it by Computer”, Pear son Education, 2008. \n3. Kanetkar Y, “Let us C”, BPB Publications, 2007. \n4. Hanly J R & Koffman E.B, “Problem Solving and Programm design in C”, Pearson \nEducation, 2009. \n \nSUPPLEMENTARY READING \n \n1. E. Balagurusamy, “Programming with ANSI-C”, Fourth Edition,2008, Tata McGraw Hill. \n2. Venugopal K. R and Prasad S. R, “Mastering ‘C’”,  Third Edition, 2008, Tata McGraw \nHill. \n3. B.W. Kernighan & D. M. Ritchie, “The C Programming Language”, Second E

In [54]:
files = [
    "/home/vasim/Downloads/PDFs/M3-R4 Programming and Problem Solving through C.pdf",
    "/home/vasim/Downloads/PDFs/M3-R4 Programming and Problem Solving through C.pdf",
]

In [55]:
def get_pdf_content(file_path: str) -> list:
    """Load a PDF file and return its content as a string."""
    loader = PyPDFLoader(file_path)
    documents = loader.load()
    return documents

def get_text_content(file_path: str) -> str:
    """Load a text file and return its content as a string."""
    with open(file_path, 'r') as file:
        content = file.read()
    return content

In [56]:
def get_retriever_of_files(
    files: list,
    model_name: str = "sentence-transformers/all-MiniLM-L6-v2",
    chunk_size: int = 1000,
    chunk_overlap: int = 200,
    search_type: str = "mmr",
    search_kwargs: dict = {"k": 1},
) -> FAISS:
    """Load multiple PDF files and create a retriever using FAISS."""
    all_documents = []
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    documents

    for file_path in files:
        if file_path.endswith('.txt'):
            documents = get_text_content(file_path)
        elif file_path.endswith('.pdf'):
            documents = get_pdf_content(file_path)
        else:
            raise ValueError(f"Unsupported file type: {file_path}")
        
        all_documents.extend(documents)
    
    all_documents = text_splitter.split_documents(all_documents)

    embeddings = HuggingFaceEmbeddings(model_name=model_name)
    vector_store = FAISS.from_documents(all_documents, embeddings)
    
    retriever = vector_store.as_retriever(search_type=search_type, search_kwargs=search_kwargs)
    
    return retriever